In [1]:
import pypdb
import os
import pandas as pd
import pickle
from pypdb.clients.pdb import pdb_client
import tqdm
from tqdm import tqdm

import gzip
import numpy as np
from Bio.PDB import *
from Bio.PDB.Polypeptide import three_to_one, is_aa

In [2]:
## This code for pdb file manipulation is taken is taken from https://github.com/compbiomed-unito/acdc-nn/blob/master/acdc_nn/util.py

def magic_open(path):
    return (gzip.open if path.endswith('.gz') else open)(path, 'rt')

def pdb2seq(pp):
    ''' pdb2seq(pp) takes a pdb_structure_chain 
    and return its sequence '''
    seq = [] # pp.get_sequence()
    reslist = []
    for ppc  in pp:
        reslist += [res for res in ppc]
        seq += [str(ppc.get_sequence())]
    return "".join(seq)

def map_pdb_pos(pp):
    ''' map_pdb_pos
    Returns two dicts seq2pdb[seq_pos], pdb2seq[pdb_pos]'''
    reslist = []
    for ppc  in pp:
        reslist += [res for res in ppc]
    seq2pdb = dict(zip( map(str,range(1,len(reslist)+1)), [str(r.get_id()[1])+r.get_id()[2].strip() for r in reslist]))
    pdb2seq = dict(zip( [str(r.get_id()[1])+r.get_id()[2].strip() for r in reslist], map(str,range(1,len(reslist)+1)) ))
    return seq2pdb, pdb2seq

def pdb2info(pdb_file, chain):
    ''' pdb2info(pdb_file) 
    Returns structure, polypeptide '''
    parser=PDBParser(QUIET=True)
    with magic_open(pdb_file) as f:
        structure = parser.get_structure('X', f)
    pchain=structure[0][chain]
    ppb=PPBuilder()
    pp = ppb.build_peptides(pchain, aa_only=False) #[0]
    return (structure, pchain, pdb2seq(pp), *map_pdb_pos(pp)) 

# S2648

In [3]:
df_S2648 = pd.read_csv('DATA/S2648.csv')

In [4]:
print('Total dataset length', len(df_S2648))
pdb_ids = list(set([t.split()[0].upper() for t in df_S2648.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 2648
Total number of different chains in dataset 132


In [5]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [6]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing s2648')

for idx in tqdm(range(len(df_S2648))):
    pdb_id = df_S2648.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_S2648.iloc[idx]['WILD_RES']
    pos = str(df_S2648.iloc[idx]['POSITION'])
    mutant_aa = df_S2648.iloc[idx]['MUTANT_RES']
    exp_ddg = df_S2648.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing s2648


 41%|████████████████████████████████████████████████████████████████▌                                                                                             | 1082/2648 [01:32<01:00, 25.84it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 41%|████████████████████████████████████████████████████████████████▋                                                                                             | 1085/2648 [01:32<01:00, 25.67it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

 44%|████████████████████████████████████████████████████████████████████▉                                                                                         | 1156/2648 [01:36<01:11, 20.74it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████                                                    | 1778/2648 [02:07<00:26, 33.23it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                           | 2185/2648 [03:00<00:09, 48.46it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 83%|██████████████████████████████████████████████████████████████████

In [7]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S2648.csv')

# S3488

In [8]:
df_1744 = pd.read_csv('DATA/Q1744.txt', sep = ' ', names = ['PDB_CHAIN', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [9]:
print('Total dataset length', len(df_1744))
pdb_ids = list(set([t.split()[0].upper() for t in df_1744.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 1744
Total number of different chains in dataset 127


In [10]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [11]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing S3488')

for idx in tqdm(range(len(df_1744))):
    pdb_id = df_1744.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_1744.iloc[idx]['WILD_RES']
    pos = str(df_1744.iloc[idx]['POSITION'])
    mutant_aa = df_1744.iloc[idx]['MUTANT_RES']
    exp_ddg = df_1744.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))
            
            mut.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            wt.append(''.join(tt))
            ddg.append(-1*exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing S3488


  1%|█▋                                                                                                                                                              | 18/1744 [00:01<02:13, 12.88it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
  1%|██                                                                                                                                                              | 22/1744 [00:01<01:52, 15.37it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
  4%|██████▉                                                           

Error for 1LVEA expected Q at position 89 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

  8%|████████████▋                                                                                                                                                  | 139/1744 [00:11<02:06, 12.67it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 10%|███████████████▏                                                                                                                                               | 166/1744 [00:13<01:19, 19.81it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 10%|███████████████▋                                                                                                                                               | 172/1744 [00:13<01:22, 19.09it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

 27%|██████████████████████████████████████████▌                                                                                                                    | 467/1744 [00:38<01:04, 19.71it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 27%|███████████████████████████████████████████▋                                                                                                                   | 479/1744 [00:38<00:57, 21.83it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

...

Error for 1LVEA expected Y at position 96 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

 76%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                     | 1326/1744 [01:54<00:27, 15.33it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 77%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                    | 1341/1744 [01:55<00:34, 11.62it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 78%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                  | 1364/1744 [01:58<00:25, 14.73it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

Error for 1LVEA expected Q at position 89 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 82%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                             | 1422/1744 [02:01<00:16, 19.24it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                            | 1432/1744 [02:02<00:13, 23.95it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 83%|██████████████████████████████████████████████████████████████████

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎  | 1715/1744 [02:28<00:01, 18.91it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 1719/1744 [02:28<00:01, 19.70it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 99%|██████████████████████████████████████████████████████████████████

In [12]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S3488.csv')

# S3421

In [13]:
df_3421 = pd.read_csv('DATA/Q3421.txt', sep = '\t', skiprows = 2, index_col=False,
                      names = ['PDB_ID', 'PDB_CHAIN', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG', 'T', 'PH', 'POS2'])

/tmp/ipykernel_48352/2764013025.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_3421 = pd.read_csv('DATA/Q3421.txt', sep = '\t', skiprows = 2, index_col=False,


In [14]:
print('Total dataset length', len(df_3421))
pdb_ids = list(set([t.split()[0].upper() for t in df_3421.PDB_ID.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 3421
Total number of different chains in dataset 148


In [15]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [16]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

verbatim_pdb_ids = {'1LVEA'}


print('Processing S3421')

for idx in tqdm(range(len(df_3421))):
    pdb_id = df_3421.iloc[idx]['PDB_ID'].upper() + df_3421.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_3421.iloc[idx]['WILD_RES']
    pos = str(df_3421.iloc[idx]['POSITION'])
    mutant_aa = df_3421.iloc[idx]['MUTANT_RES']
    exp_ddg = df_3421.iloc[idx]['EXP_DDG']
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id in verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing S3421


  7%|██████████▉                                                                                                                                                    | 236/3421 [00:40<01:53, 28.00it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 12%|███████████████████▎                                                                                                                                           | 415/3421 [00:50<02:46, 18.08it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 12%|███████████████████▍                                              

Error for 1LVEA expected Q at position 89 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

 31%|████████████████████████████████████████████████▊                                                                                                             | 1058/3421 [01:33<02:56, 13.36it/s]

Error for 1LVEA expected S at position 29 
Sequence is DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNSKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPYSFGQGTKLEIKR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69'

 43%|████████████████████████████████████████████████████████████████████▋                                                                                         | 1488/3421 [02:07<00:46, 41.62it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 44%|█████████████████████████████████████████████████████████████████████                                                                                         | 1495/3421 [02:08<00:39, 48.73it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 45%|███████████████████████████████████████████████████████████████████████▎                                                                                      | 1545/3421 [02:09<00:56, 33.26it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 46%|█████████████████████████████████████████████████████████████████████████                                                                                     | 1581/3421 [02:10<01:04, 28.72it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
 47%|██████████████████████████████████████████████████████████████████████████▋                                                                                   | 1618/3421 [02:12<01:03, 28.42it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue DDF is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
 83%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                           | 2834/3421 [03:30<00:15, 37.76it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue PLM is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: U

In [17]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/S3421.csv')

# ACDC-varibench

In [18]:
df_acdc_varibench = pd.concat([pd.read_csv(os.path.join('DATA/varibench/', f), sep = ' ',
            names = ['PDB_CHAIN', 'MUTATION', 'EXP_DDG']) for f in os.listdir('DATA/varibench/')]).drop_duplicates()

In [19]:
print('Total dataset length', len(df_acdc_varibench))
pdb_ids = list(set([t.split()[0].upper() for t in df_acdc_varibench.PDB_CHAIN.to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 1387
Total number of different chains in dataset 78


In [20]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [21]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing ACDC-varibench')

for idx in tqdm(range(len(df_acdc_varibench))):
    pdb_id = df_acdc_varibench.iloc[idx]['PDB_CHAIN'].upper()
    wild_aa = df_acdc_varibench.iloc[idx]['MUTATION'][0]
    pos = df_acdc_varibench.iloc[idx]['MUTATION'][1:-1]
    mutant_aa = df_acdc_varibench.iloc[idx]['MUTATION'][-1]
    exp_ddg = df_acdc_varibench.iloc[idx]['EXP_DDG']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    if pdb_id not in no_verbatim_pdb_ids:
        seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing ACDC-varibench


  1%|█▊                                                                                                                                                              | 16/1387 [00:00<01:45, 12.94it/s]

Error for 1AM7A expected H at position 30 
Sequence is MVEINNQRKAFLDMLAWSEGTDNGRQKTRNHGYDVIVGGELFTDYSDHPRKLVTLNPKLKSTGAGRYQLLSRWWDAYRKQLGLKDFSPKSQDAVALQQIKERGALPMIDRGDIRQAIDRCSNIWASLPGAGYGQFEHKADSLIAKFKEAGGTVR
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': 

 15%|███████████████████████▎                                                                                                                                       | 203/1387 [00:07<00:25, 46.58it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 15%|███████████████████████▊                                                                                                                                       | 208/1387 [00:08<00:29, 40.25it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

Error for 1ONCA expected M at position 22 
Sequence is EDWLTFQKKHITNTRDVDCDNIMSTNLFHCKDKNTFIYSRPEPVKAICKGIIASKNVLTTSEFYLSDCNVTSRPCKYKLKKSTNKFCVTCENQAPVHFVGVGSC
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70': '7

...

 46%|█████████████████████████████████████████████████████████████████████████▍                                                                                     | 641/1387 [00:25<00:21, 34.29it/s]

Error for 1YCCA expected P at position 80 
Sequence is TEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSGQAEGYSYTDANIKKNVLWDENNMSEYLTNPKKYIPGTKMAFGGLKKEKDRNDLITYLKKACE
Mapping is {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70'

 65%|███████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                       | 904/1387 [00:53<00:24, 20.10it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 65%|████████████████████████████████████████████████████████████████████████████████████████████████████████                                                       | 908/1387 [00:53<00:21, 22.80it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

Indexing error for 1BNIA position 108 not present in mapping {'0': '0', '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '14': '14', '15': '15', '16': '16', '17': '17', '18': '18', '19': '19', '20': '20', '21': '21', '22': '22', '23': '23', '24': '24', '25': '25', '26': '26', '27': '27', '28': '28', '29': '29', '30': '30', '31': '31', '32': '32', '33': '33', '34': '34', '35': '35', '36': '36', '37': '37', '38': '38', '39': '39', '40': '40', '41': '41', '42': '42', '43': '43', '44': '44', '45': '45', '46': '46', '47': '47', '48': '48', '49': '49', '50': '50', '51': '51', '52': '52', '53': '53', '54': '54', '55': '55', '56': '56', '57': '57', '58': '58', '59': '59', '60': '60', '61': '61', '62': '62', '63': '63', '64': '64', '65': '65', '66': '66', '67': '67', '68': '68', '69': '69', '70': '70', '71': '71', '72': '72', '73': '73', '74': '74', '75': '75', '76': '76', '77': '77', '78': '78', '79': '79'

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                          | 1153/1387 [01:03<00:08, 28.78it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                          | 1157/1387 [01:03<00:08, 27.40it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/Polypeptide.py:342: UserWarning: Assuming residue  CA is an unknown modified amino acid
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB

In [22]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/ACDC_varibench.csv')

# DeepDDG train

In [23]:
df_deepddg_train = pd.read_csv('DATA/deep_ddg_train.csv', sep = ';')

In [24]:
print('Total dataset length', len(df_deepddg_train))
pdb_ids = list(set([t.split()[0].upper() for t in df_deepddg_train['PDB ID with modifications to be made'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

Total dataset length 5444
Total number of different chains in dataset 209


In [25]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [ ]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

#no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing DeepDDG train')

for idx in tqdm(range(len(df_deepddg_train))):
    pdb_id = df_deepddg_train.iloc[idx]['PDB ID with modifications to be made'].upper()
    wild_aa = df_deepddg_train.iloc[idx]['Mutation'][0]
    pos = df_deepddg_train.iloc[idx]['Mutation'][2:-2]
    mutant_aa = df_deepddg_train.iloc[idx]['Mutation'][-1]
    exp_ddg = df_deepddg_train.iloc[idx]['ΔΔG (kcal/mol) positive is stable']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    
    pdb = PDBParser().get_structure("pdb_id[:4]", f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', chain)
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #    seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

Processing DeepDDG train


  0%|▍                                                                                                                                                             | 15/5444 [00:07<1:09:13,  1.31it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3425.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3507.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3425.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3507.
  warnings.warn(
  0%|▍                                                       

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2206.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2249.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2292.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2407.
  warnings.warn(
  0%|▊                                                                                                                                                               | 27/5444 [00:08<15:22,  5.87it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packag

  1%|█▏                                                                                                                                                              | 40/5444 [00:09<06:37, 13.61it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1513.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 1520.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 2734.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain I is discontinuous at line 2847.
  warnings.warn(
  1%|█▏                                                      

Error for 1ACB:I:F10W expected V at position 18 
Sequence is CGVPAIQPVLSGLIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSLINENWVVTAAHCGVTTSDVVVAGEFDQGSSSEKIQKLKIAKVFKNSKYNSLTINNDITLLKLSTAASFSQTVSAVCLPSASDDFAAGTTCVTTGWGLTRYANTPDRLQQASLPLLSNTNCKKYWGTKIKDAMICAGASGVSSCMGDSGGPLVCKKNGAWTLVGIVSWGSSTCSTSTPGVYARVTALVNWVQQTLAAN
Mapping is {'1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '10': '10', '11': '11', '12': '12', '13': '13', '16': '14', '17': '15', '18': '16', '19': '17', '20': '18', '21': '19', '22': '20', '23': '21', '24': '22', '25': '23', '26': '24', '27': '25', '28': '26', '29': '27', '30': '28', '31': '29', '32': '30', '33': '31', '34': '32', '35': '33', '36': '34', '37': '35', '38': '36', '39': '37', '40': '38', '41': '39', '42': '40', '43': '41', '44': '42', '45': '43', '46': '44', '47': '45', '48': '46', '49': '47', '50': '48', '51': '49', '52': '50', '53': '51', '54': '52', '55': '53', '56': '54', '57': '55', '58': '56', '59': '57', '60': '58', '61': '

...

 57%|█████████████████████████████████████████████████████████████████████████████████████████▊                                                                    | 3093/5444 [08:42<07:33,  5.18it/s]/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 6146.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 6147.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 6148.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 6149.
  warnings.warn(
/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packag

Indexing error for 1TUP position 220 not present in mapping {}


 57%|█████████████████████████████████████████████████████████████████████████████████████████▉                                                                    | 3097/5444 [08:43<04:10,  9.37it/s]

Indexing error for 1TUP position 236 not present in mapping {}


 58%|███████████████████████████████████████████████████████████████████████████████████████████▌                                                                  | 3157/5444 [09:04<27:59,  1.36it/s]

In [ ]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [float(t.replace(',','.')) for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/deepddg_train.csv')

# DeepDDG test

In [ ]:
df_deepddg_test = pd.read_csv('DATA/deep_ddg_test.csv', sep = ';')

In [ ]:
print('Total dataset length', len(df_deepddg_test))
pdb_ids = list(set([t.split()[0].upper() for t in df_deepddg_test['PDB ID with modifications to be made'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

In [ ]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [ ]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

#no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing DeepDDG test')

for idx in tqdm(range(len(df_deepddg_test))):
    pdb_id = df_deepddg_test.iloc[idx]['PDB ID with modifications to be made'].upper()
    wild_aa = df_deepddg_test.iloc[idx]['Mutation'][0]
    pos = df_deepddg_test.iloc[idx]['Mutation'][2:-2]
    mutant_aa = df_deepddg_test.iloc[idx]['Mutation'][-1]
    exp_ddg = df_deepddg_test.iloc[idx]['ΔΔG (kcal/mol) positive is stable']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    
    pdb = PDBParser().get_structure("pdb_id[:4]", f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', chain)
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #    seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

In [ ]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [float(t.replace(',','.')) for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/deepddg_test.csv')

# Ssym

In [ ]:
df_ssym = pd.read_csv('DATA/s_sym.txt', sep= ' ', names = ['PDB_ID', '_', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [ ]:
print('Total dataset length', len(df_ssym))
pdb_ids = list(set([t.split()[0].upper() for t in df_ssym['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

In [ ]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [ ]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing Ssym')

for idx in tqdm(range(len(df_ssym))):
    pdb_id = df_ssym.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_ssym.iloc[idx]['WILD_RES']
    pos = str(df_ssym.iloc[idx]['POSITION'])
    mutant_aa = df_ssym.iloc[idx]['MUTANT_RES']
    exp_ddg = df_ssym.iloc[idx]['EXP_DDG']
    
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #  seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

In [ ]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/ssym.csv')

In [ ]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/ssym_r.csv')

# Myoglobin

In [ ]:
df_myoglobin = pd.read_csv('DATA/myoglobin.txt', sep= ' ', names = ['PDB_ID', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [ ]:
print('Total dataset length', len(df_myoglobin))
pdb_ids = list(set([t.split()[0].upper() for t in df_myoglobin['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

In [ ]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [ ]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

#no_verbatim_pdb_ids = {'1C9OA', '1VQBA'}


print('Processing myoglobin')

for idx in tqdm(range(len(df_myoglobin))):
    pdb_id = df_myoglobin.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_myoglobin.iloc[idx]['WILD_RES']
    pos = str(df_myoglobin.iloc[idx]['POSITION'])
    mutant_aa = df_myoglobin.iloc[idx]['MUTANT_RES']
    exp_ddg = df_myoglobin.iloc[idx]['EXP_DDG']
    
    #if pdb_id!= '1CLWA':
    #    continue
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    #if pdb_id not in no_verbatim_pdb_ids:
    #  seq2pdb_pos = {str(i):str(i) for i in range(len(sequence))}
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

In [ ]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/myoglobin.csv')

In [ ]:
pd.DataFrame({'wt_seq': mut, 
              'mut_seq': wt ,
              'ddg': ddg, 
              'pdb_id': pdb_ids, 
              'mut_info': [t[-1] + t[1:-1] + t[0] for t in mut_infos],
              'pos': poss}).to_csv('DATASETS/myoglobin_r.csv')

# P53

In [ ]:
df_p53 = pd.read_csv('DATA/p53.txt', sep= ' ', names = ['PDB_ID', 'POSITION', 'WILD_RES', 'MUTANT_RES', 'EXP_DDG'])

In [ ]:
print('Total dataset length', len(df_p53))
pdb_ids = list(set([t.split()[0].upper() for t in df_p53['PDB_ID'].to_list()]))
print('Total number of different chains in dataset', len(pdb_ids))

In [ ]:
for pdb_id in pdb_ids:
    if not os.path.isfile(f"PDB/{pdb_id[:4]}.pdb"):
        with open(f"PDB/{pdb_id[:4]}.pdb", "w") as fh:
            fh.write(pdb_client.get_pdb_file(f"{pdb_id[:4]}", compression=False))

In [ ]:
wt = []
mut = []
ddg = []
pdb_ids = []
mut_infos = []
poss = []

print('Processing p53')

for idx in tqdm(range(len(df_myoglobin))):
    pdb_id = df_myoglobin.iloc[idx]['PDB_ID'].upper()
    wild_aa = df_myoglobin.iloc[idx]['WILD_RES']
    pos = str(df_myoglobin.iloc[idx]['POSITION'])
    mutant_aa = df_myoglobin.iloc[idx]['MUTANT_RES']
    exp_ddg = df_myoglobin.iloc[idx]['EXP_DDG']
    
        
    
    pdb = PDBParser().get_structure(pdb_id[:4], f'PDB/{pdb_id[:4]}.pdb')
    chain = next(pdb.get_chains()).get_id()
        
    _, _, sequence, pdb2seq_pos, seq2pdb_pos = pdb2info(f'PDB/{pdb_id[:4]}.pdb', pdb_id[-1])
    
    
    if pos not in seq2pdb_pos:
        print(f'Indexing error for {pdb_id} position {pos} not present in mapping {seq2pdb_pos}')
        
    else:
        if sequence[int(seq2pdb_pos[pos])-1]!=wild_aa:
            print(f'Error for {pdb_id} expected {wild_aa} at position {pos} ')
            print(f'Sequence is {sequence}')
            print(f'Mapping is {seq2pdb_pos}')
        
        else:
            wt.append(sequence)
            tt = list(sequence)
            tt[int(seq2pdb_pos[pos])-1] = mutant_aa
            poss.append(int(seq2pdb_pos[pos])-1)
            mut.append(''.join(tt))
            ddg.append(exp_ddg)
            pdb_ids.append(pdb_id)
            mut_infos.append(str(wild_aa) + pos + str(mutant_aa))

In [ ]:
pd.DataFrame({'wt_seq': wt, 
              'mut_seq': mut ,
              'ddg': [-t for t in ddg], 
              'pdb_id': pdb_ids, 
              'mut_info': mut_infos,
              'pos': poss}).to_csv('DATASETS/p53.csv')